In [1]:
# Dependencies
import numpy as np
import pandas as pd
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect


In [2]:
engine = create_engine("sqlite:///hawaii.sqlite")

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [3]:
# Create our session (link) from Python to the DB
session = Session(engine)

## D1: Determine the Summary Statistics for June

In [4]:
# 1. Import the sqlalchemy extract function.
from sqlalchemy import extract

# 2. Write a query that filters the Measurement table to retrieve the temperatures for the month of June. 
results=session.query(Measurement.tobs).filter(extract('month',Measurement.date)==6).all()

In [5]:
#  3. Convert the June temperatures to a list.
june_temps = list(np.ravel(results))


In [6]:
# 4. Create a DataFrame from the list of temperatures for the month of June. 

june_temp_df = pd.DataFrame(june_temps,columns = ["June Temps"])

june_temp_df

,June Temps
0,78.0
1,76.0
2,78.0
3,76.0
4,77.0
...,...
1695,79.0
1696,74.0
1697,74.0
1698,76.0


In [7]:
# 5. Calculate and print out the summary statistics for the June temperature DataFrame.
june_temp_df.describe()

,June Temps
count,1700.000000
mean,74.944118
std,3.257417
min,64.000000
25%,73.000000
50%,75.000000
75%,77.000000
max,85.000000


## D2: Determine the Summary Statistics for December

In [8]:
# 6. Write a query that filters the Measurement table to retrieve the temperatures for the month of December.
results=session.query(Measurement.tobs).filter(extract('month',Measurement.date)==12).all()

In [9]:
# 7. Convert the December temperatures to a list.
dec_temps = list(np.ravel(results))

In [10]:
# 8. Create a DataFrame from the list of temperatures for the month of December. 
dec_temp_df = pd.DataFrame(dec_temps,columns = ["Dec Temps"])

dec_temp_df

,Dec Temps
0,76.0
1,74.0
2,74.0
3,64.0
4,64.0
...,...
1512,71.0
1513,71.0
1514,69.0
1515,65.0


In [11]:
# 9. Calculate and print out the summary statistics for the Decemeber temperature DataFrame.
dec_temp_df.describe()

,Dec Temps
count,1517.000000
mean,71.041529
std,3.745920
min,56.000000
25%,69.000000
50%,71.000000
75%,74.000000
max,83.000000


## D3: Two Addtional Querries

In [12]:
# Review all columns in Measurement Table with inspector tool

inspector = inspect(engine)

# Set column output to mesurement_columns
Measurement_columns = inspector.get_columns('measurement')

# For loop to create list of column_names
meas_column_names = []

for name in Measurement_columns:
    meas_column_names.append(name['name'])
    
print(meas_column_names)

['id', 'station', 'date', 'prcp', 'tobs']


In [13]:
# Review all columns in Station Table with inspector tool

inspector = inspect(engine)

# Set column output to mesurement_columns
station_columns = inspector.get_columns('station')

# For loop to create list of column_names
station_column_names = []

for name in station_columns:
    station_column_names.append(name['name'])
    
print(station_column_names)

['id', 'station', 'name', 'latitude', 'longitude', 'elevation']


### D3: Average Temperature and Average Precip by Station Decmeber 

In [14]:
# Create DataFrame for Station Table
con=engine
station_df = pd.read_sql_query("SELECT * FROM station",con)
station_df.drop(columns=['id','latitude','longitude'],inplace=True)
station_df

,station,name,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",3.0
1,USC00513117,"KANEOHE 838.1, HI US",14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",7.0
3,USC00517948,"PEARL CITY, HI US",11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",306.6
5,USC00519523,"WAIMANALO EXPERIMENTAL FARM, HI US",19.5
6,USC00519281,"WAIHEE 837.5, HI US",32.9
7,USC00511918,"HONOLULU OBSERVATORY 702.2, HI US",0.9
8,USC00516128,"MANOA LYON ARBO 785.2, HI US",152.4


In [15]:
# Create Data Frame for Average Temp and Precipitation by Station ID for December
dec_results=session.query(Measurement.station,func.avg(Measurement.tobs),func.avg(Measurement.prcp))\
    .filter(extract('month',Measurement.date)==12).group_by(Measurement.station).all()

dec_station_weather_df = pd.DataFrame(dec_results,columns=['station','Avg Temp','Avg Precip'])
dec_station_weather_df

,station,Avg Temp,Avg Precip
0,USC00511918,69.684211,0.138146
1,USC00513117,71.069444,0.203241
2,USC00514830,73.224719,0.154966
3,USC00516128,69.291262,0.507005
4,USC00517948,71.834862,0.152727
5,USC00518838,72.421053,0.638182
6,USC00519281,69.903226,0.244931
7,USC00519397,71.109524,0.075314
8,USC00519523,72.433333,0.162010


In [16]:
# Merge Station and Mesurement Table to attach location name to weather data
dec_surfs_up_df = station_df.merge(dec_station_weather_df, how='right',on=['station','station'])

# Format Data Frame Sort Columns, change decimal places for calculated fields
dec_surfs_up_df.sort_values(by=['Avg Temp'],ascending=False,inplace=True)
dec_surfs_up_df['Avg Temp']=dec_surfs_up_df['Avg Temp'].map("{:.1f}".format)
dec_surfs_up_df['Avg Precip']=dec_surfs_up_df['Avg Precip'].map("{:.3f}".format)

# Change Column Names and view DataFrame choose only stations below 50 feet in elevation
dec_surfs_up_df.columns=['Station Id',"Station Name","Elevation","Avg Temp","Avg Precip"]
dec_surfs_up_df= dec_surfs_up_df[dec_surfs_up_df['Elevation']<50]
dec_surfs_up_df

,Station Id,Station Name,Elevation,Avg Temp,Avg Precip
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",7.0,73.2,0.155
8,USC00519523,"WAIMANALO EXPERIMENTAL FARM, HI US",19.5,72.4,0.162
4,USC00517948,"PEARL CITY, HI US",11.9,71.8,0.153
7,USC00519397,"WAIKIKI 717.2, HI US",3.0,71.1,0.075
1,USC00513117,"KANEOHE 838.1, HI US",14.6,71.1,0.203
6,USC00519281,"WAIHEE 837.5, HI US",32.9,69.9,0.245
0,USC00511918,"HONOLULU OBSERVATORY 702.2, HI US",0.9,69.7,0.138


### D3: Average Temperature and Average Precip by Station June 

In [17]:
# Create Data Frame for Average Temp and Precipitation by Station ID for June
june_results=session.query(Measurement.station,func.avg(Measurement.tobs),func.avg(Measurement.prcp)).filter(extract('month',Measurement.date)==6).group_by(Measurement.station).all()

june_station_weather_df = pd.DataFrame(june_results,columns=['station','Avg Temp','Avg Precip'])
june_station_weather_df

,station,Avg Temp,Avg Precip
0,USC00511918,74.139394,0.015157
1,USC00513117,74.050847,0.118248
2,USC00514830,76.005376,0.114192
3,USC00516128,71.937220,0.495748
4,USC00517948,76.655405,0.057975
5,USC00518838,73.394737,0.094615
6,USC00519281,73.271186,0.151525
7,USC00519397,77.559322,0.022661
8,USC00519523,76.668103,0.050044


In [18]:
# Merge Station and Mesurement Table to attach location name to weather data
june_surfs_up_df = station_df.merge(june_station_weather_df, how='right',on=['station','station'])

# Format Data Frame Sort Columns, change decimal places for calculated fields
june_surfs_up_df.sort_values(by=['Avg Temp'],ascending=False,inplace=True)
june_surfs_up_df['Avg Temp']=june_surfs_up_df['Avg Temp'].map("{:.1f}".format)
june_surfs_up_df['Avg Precip']=june_surfs_up_df['Avg Precip'].map("{:.3f}".format)

# Change Column Names and view DataFrame
june_surfs_up_df.columns = ['Station Id',"Station Name","Elevation","Avg Temp","Avg Precip"]
june_surfs_up_df= june_surfs_up_df[june_surfs_up_df['Elevation']<50]
june_surfs_up_df

,Station Id,Station Name,Elevation,Avg Temp,Avg Precip
7,USC00519397,"WAIKIKI 717.2, HI US",3.0,77.6,0.023
8,USC00519523,"WAIMANALO EXPERIMENTAL FARM, HI US",19.5,76.7,0.050
4,USC00517948,"PEARL CITY, HI US",11.9,76.7,0.058
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",7.0,76.0,0.114
0,USC00511918,"HONOLULU OBSERVATORY 702.2, HI US",0.9,74.1,0.015
1,USC00513117,"KANEOHE 838.1, HI US",14.6,74.1,0.118
6,USC00519281,"WAIHEE 837.5, HI US",32.9,73.3,0.152
